In [13]:
import pandas as pd
import numpy as np
import html2text
import re

In [43]:
data = pd.read_csv('new_judgements.csv')

In [44]:
data.head()

,tid,covertids,publishdate,firstname,secondname,lastname,title,filename,doc,numcites,...,url,showurl,docsource,covers,citeList,citedbyList,divtype,relatedqs,courtcopy,errmsg
0,1004544.0,[],2001-08-21,a,NaN,sikri,Khandelwal Laboratories Ltd. vs Fdc Limited on...,www.manupatra.co.in/NXT/gateway.dll/highcourt1...,"<div class=""docsource_main"">Delhi High Court</...",2.0,...,NaN,True,Delhi High Court,[],"[{'tid': 257409, 'title': 'The Indian Companie...","[{'tid': 140343706, 'title': 'Fdc Limited vs M...",judgments,"[{'value': 'pharmaceutical', 'formInput': 'pha...",False,NaN
1,102042281.0,[],2014-07-01,s,NaN,sachdeva,"Stiefel Laboratories, Inc & Anr vs Ajanta Phar...",delhi/2014-07-01/CS(OS)--2373-2013,"<div class=""docsource_main"">Delhi High Court</...",9.0,...,http://lobis.nic.in/dhc/SAS/judgement/01-07-20...,True,Delhi High Court,[],"[{'tid': 1181080, 'title': 'Century Traders vs...","[{'tid': 117371033, 'title': 'M/S Deelux Cable...",judgments,"[{'value': 'trademark', 'formInput': 'trademar...",True,NaN
2,1025815.0,[],1999-08-24,a,NaN,kapadia,Cadbury India Limited vs Sm Dyechem Limited on...,www.manupatra.co.in/NXT/gateway.dll/HighCourt3...,"<div class=""docsource_main"">Gujarat High Court...",20.0,...,NaN,True,Gujarat High Court,[],"[{'tid': 1353758, 'title': 'The Companies Act,...","[{'tid': 1965135, 'title': 'Torrent Pharmaceut...",judgments,"[{'value': 'picnic', 'formInput': 'picnic'}, {...",False,NaN
3,1062207.0,[],2001-03-21,d,NaN,deshmukh,Agromore (P) Ltd. And Anr. vs Chembond Chemica...,www.manupatra.co.in/NXT/gateway.dll/highcourt1...,"<div class=""docsource_main"">Bombay High Court<...",20.0,...,NaN,True,Bombay High Court,[],"[{'tid': 181078, 'title': 'Section 33 in The C...","[{'tid': 1373362, 'title': 'Ms. Supriya Prabhu...",judgments,"[{'value': 'trade mark use', 'formInput': 'tra...",False,NaN
4,106681665.0,[],1983-09-30,d,NaN,desai.,Vishwa Mitter Of Vijay Bharat ... vs O.P. Podd...,www.manupatra.co.in/NXT/gateway.dll/sc/1980-20...,"<div class=""docsource_main"">Supreme Court of I...",17.0,...,NaN,True,Supreme Court of India,[],"[{'tid': 1017213, 'title': 'The Trade Marks Ac...","[{'tid': 47858029, 'title': 'A.C. Narayanan vs...",judgments,"[{'value': 'beedi', 'formInput': 'beedi'}, {'v...",False,NaN


In [4]:
data.columns

Index(['tid', 'covertids', 'publishdate', 'firstname', 'secondname',
       'lastname', 'title', 'filename', 'doc', 'numcites', 'numcitedby',
       'covertitles', 'url', 'showurl', 'docsource', 'covers', 'citeList',
       'citedbyList', 'divtype', 'relatedqs', 'courtcopy', 'errmsg'],
      dtype='object')

In [19]:
len(data)

96

In [6]:
df = pd.read_csv('casemine.csv')

In [7]:
df.head()

,Judgement Name,IndianKanoon Link,Similar Judgement,IndianKanoon Link.1,Rank
0,Wander Ltd. And Another v. Antox India P. Ltd. .,https://indiankanoon.org/doc/330608/,Astrazeneca Uk Ltd. v. Orchid Chemicals & Phar...,https://indiankanoon.org/doc/1509899/,1
1,Wander Ltd. And Another v. Antox India P. Ltd. .,https://indiankanoon.org/doc/330608/,S.M Dyechem Ltd. v. Cadbury (India) Ltd. .,https://indiankanoon.org/doc/1025815/,2
2,Wander Ltd. And Another v. Antox India P. Ltd. .,https://indiankanoon.org/doc/330608/,Veerumal Praveen Kumar v. Needle Industries (I...,https://indiankanoon.org/doc/1945028/,3
3,Wander Ltd. And Another v. Antox India P. Ltd. .,https://indiankanoon.org/doc/330608/,N.R Dongre And Others v. Whirlpool Corporation...,https://indiankanoon.org/doc/850381/,4
4,Wander Ltd. And Another v. Antox India P. Ltd. .,https://indiankanoon.org/doc/330608/,Pfizer Products Inc. v. Rajesh Chopra,https://indiankanoon.org/doc/1674695/,5


In [22]:
def get_id(link):
    regex = re.compile(r'[0-9]+')
    result = regex.search(link)
    
    if(result):
        return result.group()
    else:
        return ""

In [23]:
doclink = {}


for i in range(df.shape[0]):
    m_j = df.iloc[i,0]
    s_j = df.iloc[i,2]
    m_j_l = df.iloc[i,1]
    s_j_l = df.iloc[i,3]
    
    if m_j not in doclink.keys() and not pd.isna(m_j_l):
        doclink[m_j] = get_id(str(m_j_l))
    if s_j not in doclink.keys() and not pd.isna(m_j_l):
        doclink[s_j] = get_id(str(s_j_l))
    

In [24]:
doclink

{'Wander Ltd. And Another v. Antox India P. Ltd. .': '330608',
 'Astrazeneca Uk Ltd. v. Orchid Chemicals & Pharmaceuticals Ltd.': '1509899',
 'S.M Dyechem Ltd. v. Cadbury (India) Ltd. .': '1025815',
 'Veerumal Praveen Kumar v. Needle Industries (India) Ltd. And Anr.': '1945028',
 'N.R Dongre And Others v. Whirlpool Corporation And Another': '850381',
 'Pfizer Products Inc. v. Rajesh Chopra': '1674695',
 'Mahendra & Mahendra Paper Mills Ltd. v. Mahindra & Mahindra Ltd. .': '67204010',
 'Lupin Limited v. Johnson & Johnson': '19267666',
 'Rangoon Chemical Works Pvt. Ltd. v. Royal Medicals': '',
 'M/S Alkem Laboratories Ltd. Petitioners v. Mega International (P) Ltd. S': '',
 'Wockhardt Limited v. Torrent Pharmaceuticals Limited And Another': '',
 'Cadila Health Care Ltd. v. Cadila Pharmaceuticals Ltd. .': '1114158',
 'Satilila Charitable Society And Anr. v. Skyline Education Institute (I) (P) Ltd.': '',
 'Rajendra Sadanand Rasal v. Naranppa Thenukyya Kotian And Others S': '',
 'Purshottam

In [25]:
len(doclink)

171

In [30]:
doc_with_id = pd.DataFrame(columns = ["name","id"])

In [31]:
counter = 0
for i in doclink.keys():
    if(doclink[i] != ''):
        row = [i , doclink[i]]
        doc_with_id.loc[counter] = row
        counter = counter + 1

In [32]:
doc_with_id.head()

,name,id
0,Wander Ltd. And Another v. Antox India P. Ltd. .,330608
1,Astrazeneca Uk Ltd. v. Orchid Chemicals & Phar...,1509899
2,S.M Dyechem Ltd. v. Cadbury (India) Ltd. .,1025815
3,Veerumal Praveen Kumar v. Needle Industries (I...,1945028
4,N.R Dongre And Others v. Whirlpool Corporation...,850381


In [33]:
len(doc_with_id)

95

In [34]:
doc_with_id.to_csv("Judgement_with_id.csv",index=False)

In [35]:
temp = dict(zip(doc_with_id.name,doc_with_id.id))

In [36]:
temp

{'Wander Ltd. And Another v. Antox India P. Ltd. .': '330608',
 'Astrazeneca Uk Ltd. v. Orchid Chemicals & Pharmaceuticals Ltd.': '1509899',
 'S.M Dyechem Ltd. v. Cadbury (India) Ltd. .': '1025815',
 'Veerumal Praveen Kumar v. Needle Industries (India) Ltd. And Anr.': '1945028',
 'N.R Dongre And Others v. Whirlpool Corporation And Another': '850381',
 'Pfizer Products Inc. v. Rajesh Chopra': '1674695',
 'Mahendra & Mahendra Paper Mills Ltd. v. Mahindra & Mahindra Ltd. .': '67204010',
 'Lupin Limited v. Johnson & Johnson': '19267666',
 'Cadila Health Care Ltd. v. Cadila Pharmaceuticals Ltd. .': '1114158',
 'Purshottam Vishandas Raheja And Another v. Shrichand Vishandas Raheja (Dead) Through Lrs. And Others': '81162651',
 'Fedders North American v. Show Line': '1269197',
 'Cadila Healthcare Limited v. Cadila Pharmaceuticals Limited': '1114158',
 'Ramesh Khatanmal Lulla v. Mohammad Yusuf Abdul Gaffar': '70200400',
 'Cadbury India Ltd. v. Neeraj Food Products': '652828',
 'Cadbury India Li

In [46]:
for i in range(data.shape[0]):
    #d_id = temp[data.iloc[i,6]]
    f = open("new_text_files_html/"+str(data.iloc[i,6])+".html","w+",encoding="utf-8")
    f.write(data.iloc[i,8])

TypeError: write() argument must be str, not float